# **Integración de Datos**
    Busca los datos del Censo de Población y Vivienda 2020 por municipio y los indicadores de pobreza generados por Coneval en el año 2020. Integra estos archivos en un sólo dataframe, de acuerdo al municipio. Elimina los indicadores de Coneval que no están asociados a tu reto. Verifica el número de registros y columnas del archivo. El número de registros del archivo debe ser igual al número inicial de municipios (2469) y el número de columnas el número inicial más las 2 que agregaste de los indicadores seleccionados.

In [64]:
import pandas as pd
import numpy as np

In [65]:
# Load the data
INEGI = pd.read_csv('datosINEGI.csv')
CONEVAL = pd.read_csv('datosCONEVAL.csv', encoding='latin1')

# Filter rows where NOM_LOC == "Total del Municipio"
INEGI_total = INEGI[INEGI['NOM_LOC'] == "Total del Municipio"]

/var/folders/nq/zht86vzj6n5g3py55h5xd7xr0000gn/T/ipykernel_36180/3852254959.py:2: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  INEGI = pd.read_csv('datosINEGI.csv')


In [66]:
print(INEGI.shape)
INEGI.head()


(195662, 286)


,ENTIDAD,NOM_ENT,MUN,NOM_MUN,LOC,NOM_LOC,LONGITUD,LATITUD,ALTITUD,POBTOT,...,VPH_CEL,VPH_INTER,VPH_STVP,VPH_SPMVPI,VPH_CVJ,VPH_SINRTV,VPH_SINLTC,VPH_SINCINT,VPH_SINTIC,TAMLOC
0,0,Total nacional,0,Total nacional,0,Total nacional,NaN,NaN,NaN,126014024,...,30775898,18307193,15211306,6616141,4047100,1788552,3170894,15108204,852871,*
1,0,Total nacional,0,Total nacional,9998,Localidades de una vivienda,NaN,NaN,NaN,250354,...,47005,8385,18981,1732,1113,12775,14143,51293,7154,*
2,0,Total nacional,0,Total nacional,9999,Localidades de dos viviendas,NaN,NaN,NaN,147125,...,25581,5027,11306,971,708,8247,10065,29741,5283,*
3,1,Aguascalientes,0,Total de la entidad Aguascalientes,0,Total de la Entidad,NaN,NaN,NaN,1425607,...,359895,236003,174089,98724,70126,6021,15323,128996,1711,*
4,1,Aguascalientes,0,Total de la entidad Aguascalientes,9998,Localidades de una vivienda,NaN,NaN,NaN,3697,...,732,205,212,48,41,39,62,530,20,*


In [67]:
print(CONEVAL.shape)
CONEVAL.tail()

(2469, 37)


,clave_entidad,entidad_federativa,clave_municipio,municipio,poblacion,pobreza,pobreza_pob,pobreza_e,pobreza_e_pob,pobreza_m,...,ic_ali,ic_ali_pob,carencias,carencias_pob,carencias3,carencias3_pob,plp,plp_pob,plp_e,plp_e_pob
2464,32,Zacatecas,32054,Villa Hidalgo,"20,674",74.9,"15,483",9.0,"1,863",65.9,...,29.0,"5,985",91.6,"18,940",19.5,"4,039",79.2,"16,377",41.5,"8,574"
2465,32,Zacatecas,32055,Villanueva,"32,969",50.6,"16,673",4.3,"1,415",46.3,...,16.1,"5,321",80.9,"26,672",14.4,"4,735",58.9,"19,407",23.2,"7,634"
2466,32,Zacatecas,32056,Zacatecas,"149,864",23.4,"35,054",2.6,"3,822",20.8,...,14.1,"21,190",50.3,"75,367",7.5,"11,199",34.6,"51,866",11.0,"16,491"
2467,32,Zacatecas,32057,Trancoso,"20,270",63.9,"12,962",6.4,"1,290",57.6,...,26.0,"5,268",81.2,"16,462",14.4,"2,910",74.5,"15,098",37.9,"7,678"
2468,32,Zacatecas,32058,Santa María de la Paz,"2,792",48.0,"1,339",2.4,68,45.5,...,13.6,380,80.9,"2,258",9.0,251,55.4,"1,546",19.5,545


In [68]:
print(INEGI_total.shape)
INEGI_total.head()

(2469, 286)


,ENTIDAD,NOM_ENT,MUN,NOM_MUN,LOC,NOM_LOC,LONGITUD,LATITUD,ALTITUD,POBTOT,...,VPH_CEL,VPH_INTER,VPH_STVP,VPH_SPMVPI,VPH_CVJ,VPH_SINRTV,VPH_SINLTC,VPH_SINCINT,VPH_SINTIC,TAMLOC
6,1,Aguascalientes,1,Aguascalientes,0,Total del Municipio,NaN,NaN,NaN,948990,...,251719,178619,130290,80951,56131,3299,7293,74227,731,*
574,1,Aguascalientes,2,Asientos,0,Total del Municipio,NaN,NaN,NaN,51536,...,10682,4526,3882,596,556,377,1447,7206,177,*
749,1,Aguascalientes,3,Calvillo,0,Total del Municipio,NaN,NaN,NaN,58250,...,13666,6553,4749,1382,1352,443,904,8061,138,*
918,1,Aguascalientes,4,Cosío,0,Total del Municipio,NaN,NaN,NaN,17000,...,3424,1741,1664,223,191,139,414,1912,50,*
985,1,Aguascalientes,5,Jesús María,0,Total del Municipio,NaN,NaN,NaN,129929,...,31408,19920,13483,9296,6582,480,1150,11447,148,*


In [69]:
# Make MUN in INEGI_total be ENTIDAD * 1000 + MUN
INEGI_total['MUN'] = INEGI_total['ENTIDAD'] * 1000 + INEGI_total['MUN']

# Drop columns that are not needed from CONEVAL(only columns needed are 'clave_municipio', 'pobreza_m_pob', "ic_asalud_pob")
CONEVAL = CONEVAL[['clave_municipio', 'pobreza_m_pob', 'ic_asalud_pob']]

# MERGE INEGI_total and CONEVAL from MUN and clave_municipio
merged = pd.merge(INEGI_total, CONEVAL, left_on='MUN', right_on='clave_municipio')
# Drop clave_municipio
merged = merged.drop(columns=['clave_municipio'])

print(merged.shape)
merged.tail()





(2469, 288)


/var/folders/nq/zht86vzj6n5g3py55h5xd7xr0000gn/T/ipykernel_36180/2829535602.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  INEGI_total['MUN'] = INEGI_total['ENTIDAD'] * 1000 + INEGI_total['MUN']


,ENTIDAD,NOM_ENT,MUN,NOM_MUN,LOC,NOM_LOC,LONGITUD,LATITUD,ALTITUD,POBTOT,...,VPH_STVP,VPH_SPMVPI,VPH_CVJ,VPH_SINRTV,VPH_SINLTC,VPH_SINCINT,VPH_SINTIC,TAMLOC,pobreza_m_pob,ic_asalud_pob
2464,32,Zacatecas,32054,Villa Hidalgo,0,Total del Municipio,NaN,NaN,NaN,19446,...,1090,109,106,160,744,3569,80,*,"13,620","3,731"
2465,32,Zacatecas,32055,Villanueva,0,Total del Municipio,NaN,NaN,NaN,31558,...,5181,643,569,306,746,4252,101,*,"15,259","10,274"
2466,32,Zacatecas,32056,Zacatecas,0,Total del Municipio,NaN,NaN,NaN,149607,...,28630,11944,7639,681,1185,8783,129,*,"31,232","29,245"
2467,32,Zacatecas,32057,Trancoso,0,Total del Municipio,NaN,NaN,NaN,20455,...,1854,204,185,130,677,3274,59,*,"11,671","3,170"
2468,32,Zacatecas,32058,Santa María de la Paz,0,Total del Municipio,NaN,NaN,NaN,2767,...,427,37,23,59,114,542,32,*,"1,271",399
